<a href="https://colab.research.google.com/github/R0N3ldrt/Thesis/blob/main/New_Attack_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [28]:
# Importing necesary libraries
# Libraries for correct code execution 

import os, time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import pickle
import random
import csv
import re
import array
import seaborn as sns
import scipy.stats as st

from tqdm import tqdm
from functools import reduce
from random import random, gauss
from math import modf, pi, cos, sin, sqrt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
from plotly.subplots import make_subplots
from scipy.signal import savgol_filter
from scipy.stats.stats import pearsonr
from scipy.stats import spearmanr

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

from keras.models import Sequential
from keras.layers import Dense,Conv2D, Flatten
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping
from sklearn.mixture import GaussianMixture
import os, time, math, csv, joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.mixture import GaussianMixture

import os, time, math, csv, joblib
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.mixture import GaussianMixture

import warnings
warnings.filterwarnings("ignore")

sns.set_theme()

# Load Enviroment

In [2]:
# BLOQUE PARA USAR DESDE COLAB

# Google drive loading as work station for local-usage of the files.
from google.colab import drive
drive.mount('/content/gdrive',force_remount= True)

#-----------------------------------------------------------------------------

Mounted at /content/gdrive


In [3]:
# Lista para cambiar los paths rapido.
workers = ["Ronald", "Local"]

# Change the number to change the paths.
worker = workers[0]

if worker == "Ronald":
  path = "/content/gdrive/MyDrive/Thesis_Workstation/ANN_dataset"
else: path = os.getcwd()

# Pre-procesing

## Get working df

In [4]:
def prime_number_finder(stop_gap, total_num_of_data):
  prime_nums = []
  i=2
  while i <= stop_gap:
    if (total_num_of_data % i==0):
      prime_nums.append(i)
      break
    i+=1
  return prime_nums

def aggreagator_v2(df, stop_gap=50):
  total_num_of_data = df.shape[1]
  arr_prime = prime_number_finder(stop_gap, total_num_of_data)
  #agg_num = np.max(arr_prime)
  agg_num = arr_prime[0]

  # Creating new df of aggregate values
  agg_df = pd.DataFrame()

  mid_point = df.shape[1]/2

  agg=0
  loop_cnt=0
  init_column_cnt = 5
  while agg<=total_num_of_data:
    loop_cnt+=1
    # Obtain current last columns stop
    agg=agg_num+init_column_cnt
    # Select working columns
    new_df = df[df.columns[init_column_cnt:agg]]

    init_column_cnt += (agg_num)

    headers = list(new_df.columns.values) 
    if loop_cnt <= mid_point:
      # Get average of freq values for new header
      new_header_name  = headers[-1]
    else:
      new_header_name = headers[0]
    
    # Add aggragated values to new df
    agg_df[new_header_name] = new_df.mean(axis = 1)
  return agg_df

def remove_tail(df, cut_val, sample_id):
  columns_selected = []
  old_val = 0
  delta = 0
  mid_point = int(df.shape[1]/2)
  cols_headers  = list(df.columns.values)

  for i in range(0, df.shape[1]):
    if i == 0:
      cell_val = df.iloc[[0], i]
      old_val = cell_val[sample_id]
    else:
      cell_val = df.iloc[[0], i]
      val = cell_val[sample_id]
      delta = abs(old_val-val)
      old_val = val
      if delta > cut_val:
        if i <= mid_point:
          col_name_selected = cols_headers[i+2]
        else:
          col_name_selected = cols_headers[i-2]
          col_name_selected = round(float(col_name_selected), 5)
        columns_selected.append(col_name_selected)

  return columns_selected

# Removing tails conducting multiples loops with variable cutoff value until we get only two columns
def remove_tail_main(df, cut_val, sample_id):
  old_drop_cols = []
  drop_cols = remove_tail(df, cut_val, sample_id)
  while len(drop_cols) != 2:  
    # If len of columns to be dropped are 0 the select the 2 values smaller and bigger of the previous iteration
    if len(drop_cols) == 0 or len(drop_cols) == 1:
      drop_cols = [np.min(old_drop_cols), np.max(old_drop_cols)]
      break
    else:
      old_drop_cols = drop_cols
      drop_cols = remove_tail(df, cut_val, sample_id)
      cut_val += 2
  return drop_cols

def get_working_data_idx_v2(df_mean_sample, sample_id, cut_val = 0):

  df = df_mean_sample.iloc[[sample_id - 1]]

  agg_df = aggreagator_v2(df)
  
  col_vals = remove_tail_main(df, cut_val=cut_val, sample_id=sample_id)

  left_index_no = df.columns.get_loc(col_vals[0])
  rigth_index_no = df.columns.get_loc(col_vals[1])

  return left_index_no, rigth_index_no

def cut_off_idx(df): # return tuple with cut-off values (index_left_side, index_rigth_side)
  df_mean_sample = df.groupby(['PBRS_id']).mean()
  df_mean_sample = df_mean_sample.drop(['row', 'Channels', 'Distance_km', 'power_dBm'], axis = 1)

  cut_points = {}
  print('Calculating the cutoff values:')
  for sample in tqdm(df_mean_sample.index):
    left_index_no, rigth_index_no = get_working_data_idx_v2(df_mean_sample, sample_id = sample)
    cut_points[sample] = (left_index_no, rigth_index_no)

  left_cut_off = np.min([v[0] for k, v in cut_points.items()])
  rigth_cut_off = np.max([v[1] for k, v in cut_points.items()])

  cut_point = (left_cut_off, rigth_cut_off)
  return cut_point

### Cutoff main

In [5]:
def select_working_data(df, cut_point): # implement cut_off values and Smoothing original df after cutt-of
  extra_info_df = df.iloc[:, 0:6]
  data_df = df.iloc[:, cut_point[0]+6:cut_point[1]+6]
  smoothed_data_frame = pd.DataFrame(savgol_filter(data_df, window_length = 5, polyorder = 2))
  
  # Adding back headers to the smoothed data
  rename_col = {}
  cnt = 0
  for col in data_df.columns:
    h_col = round(float(col), 5)
    rename_col[cnt] = h_col
    cnt += 1
  smoothed_data_frame.rename(columns=rename_col, inplace=True)

  working_df = pd.merge(extra_info_df, smoothed_data_frame, left_index=True, right_index=True)
  return working_df
  
def main_cutoff(input_path):
  print('Reading data from file.')
  df = pd.read_excel(input_path, sheet_name = "Sheet1", skiprows=1)
  df.iloc[0].fillna(method='bfill', inplace=True)
  df.iloc[1:df.shape[0]].fillna(method='pad', inplace=True)
  # Adding missing PBRS_id
  snippet = []
  val_cnt = 1
  idx = 0
  for sample in range(1, 101):
    for sample_id in range(0, 25):
      df.at[idx, 'PBRS_id'] = val_cnt
      idx +=1
      snippet.append(val_cnt)
    val_cnt += 1
  df['PBRS_id'] = df['PBRS_id'].astype(int)

  cut_point = cut_off_idx(df)
  working_df = select_working_data(df, cut_point)

  return working_df

In [6]:
#input_path = path + "/Spectrum/16QAM_v2/Copy of dataSet_gamma0_Spectrum31MHz_Samples_16QAM_75GHz_LongHaul_input_25x81km_primeStep.xlsx"
#orking_df = main_cutoff(input_path)
#display(working_df)

#working_df.to_csv(path+"/Spectrum/NN/new_data_working_df.csv", index=False)

# Get data

In [13]:
input_path = path + "/Spectrum/NN/new_data_working_df.csv"

working_df = pd.read_csv(input_path)
working_df['Distance_km'] = working_df['Distance_km'].astype(int)

In [14]:
def frequencies_filter(df, num_selected_freq=20):
  info_df = df.iloc[:, 0:6]
  data_df = df.iloc[:, 6:df.shape[1]]
  span_val = int(data_df.shape[1]/num_selected_freq)
  for i in range(num_selected_freq):
    df2 = data_df.iloc[:, (i+1)*span_val].to_frame()
    if i+1 == 1:
      new_df = df2
    else:
      new_df = pd.merge(new_df, df2, left_index=True, right_index=True)
  new_df = info_df.join(new_df)
  
  return new_df

In [15]:
def get_data(working_df, distances, num_features = 20, span_length=80, min_dist = 0, max_dist = 3000):
  X=None
  Y=np.array([])
  dist_df = working_df.iloc[:, 3]
  freq_df = working_df.iloc[:, 6:working_df.shape[1]]
  samples_df = pd.merge(dist_df, freq_df, left_index=True, right_index=True)
  samples_df = samples_df.sort_values(by='Distance_km')
  if num_features != 'all':
    samples_df = frequencies_filter(samples_df, num_selected_freq = num_features)
  Y = samples_df.iloc[:, 0].to_numpy().astype('int').tolist()
  X = samples_df.iloc[:, 1:samples_df.shape[1]]
  
  return X, Y, samples_df

In [16]:
#input_data_path=path+"/rawData"
#distances=[i for i in range(1,26)]
distances = [x*80 for x in range(1, 26)]
X, Y, samples_filtered_df = get_data(working_df, distances, num_features = 20, span_length=80,  min_dist = 0, max_dist = 3000)

In [17]:
samples_filtered_df

,Distance_km,-39.90625,-39.875,-39.84375,-39.8125,-39.78125,-36.03125,-32.3125,-28.59375,-24.875,...,1.15625,4.875,8.59375,12.3125,16.03125,19.75,23.46875,27.1875,30.90625,34.625
0,80,-65.158556,-66.359675,-66.708836,-71.777931,-66.092081,-27.690134,-9.837106,-8.570167,-18.564278,...,-18.195032,-6.031685,-18.817579,-11.888566,-13.083380,-10.776726,-7.770254,-14.975903,-32.511409,-54.936101
125,80,-54.902319,-57.469566,-58.650167,-58.256577,-57.120871,-41.770630,-12.217276,-11.941760,-7.843633,...,-15.297383,-8.139403,-10.755553,-6.436845,-16.926361,-11.355173,-12.832245,-19.360144,-26.611831,-63.942755
975,80,-55.246289,-58.903165,-59.936123,-58.754704,-53.084186,-34.557578,-10.698894,-9.896776,-10.010543,...,-13.702482,-10.232840,-13.943233,-16.457126,-7.550473,-10.938752,-11.536323,-16.588483,-31.487828,-61.206569
1000,80,-58.769150,-60.644357,-61.903338,-66.371732,-60.697843,-29.246157,-14.221867,-10.839692,-9.182389,...,-14.797859,-8.605176,-7.879467,-16.755640,-12.121632,-19.470737,-9.953149,-5.720428,-29.343520,-58.192058
1025,80,-62.672232,-63.551920,-63.130158,-61.900052,-61.208010,-32.017701,-14.658309,-11.507879,-11.001936,...,-12.161145,-22.188481,-6.138815,-10.292705,-12.796458,-16.594342,-14.534831,-18.283760,-28.539294,-59.391250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1699,2000,-39.060418,-40.089715,-39.184312,-33.321699,-32.394581,-50.007449,-24.872696,-10.515505,-8.076762,...,-7.773010,-7.963502,-12.198785,-9.710933,-9.262409,-14.440979,-11.943268,-10.576351,-31.997584,-34.396245
1724,2000,-45.777276,-39.520381,-35.212366,-31.215854,-31.976366,-27.656348,-12.631312,-14.168680,-9.781711,...,-11.797013,-13.710833,-8.486654,-22.866049,-17.475989,-10.323883,-6.255820,-13.617403,-28.224603,-36.106239
1749,2000,-33.005394,-37.672756,-39.242985,-37.025965,-37.250896,-29.299168,-11.024047,-8.927058,-12.311278,...,-6.843578,-13.003866,-13.901596,-9.085603,-10.311707,-14.114579,-17.627094,-8.410328,-19.305808,-26.014417
2399,2000,-36.358480,-32.709908,-31.364002,-33.805751,-35.584646,-32.306799,-11.983912,-12.754049,-12.447543,...,-10.935418,-13.228988,-8.937448,-6.708680,-14.849681,-12.124128,-10.310341,-14.362797,-29.497050,-42.096482


## Attack

In [21]:
# source_df = data that is wished to be modified
# target_df = data that the source would convert

source_df = samples_filtered_df.loc[samples_filtered_df['Distance_km'] == 2000]
target_df = samples_filtered_df.loc[samples_filtered_df['Distance_km'] == 80]

In [40]:
def get_eval_data(source_df, target_df, num_eval_samples = 10):
  import random
  source_val_list = [i for i in range(1, len(source_df))]
  source_random_list = random.choices(source_val_list, k = num_eval_samples)

  target_val_list = [i for i in range(1, len(target_df))]
  target_random_list = random.choices(target_val_list, k = num_eval_samples)

  eval_source_df = source_df.iloc[source_random_list]
  eval_target_df = target_df.iloc[target_random_list]

  return eval_source_df, eval_target_df

In [51]:
eval_source_df, eval_target_df = get_eval_data(source_df, target_df, num_eval_samples = 10)

In [59]:
def get_ci(df, alpha = 0.90):
  ci_arr = []
  for i in range(1, df.shape[1]):
    eval_col = df.iloc[:,i].to_numpy()
    source_ci = st.t.interval(alpha=alpha, df=len(eval_col)-1, loc=np.mean(eval_col), scale=st.sem(eval_col))
    ci_arr.append(source_ci)
  return ci_arr

In [60]:
ci_arr = get_ci(eval_source_df, alpha = 0.90)
ci_arr

[(-41.91108832729236, -33.038700142638405),
 (-38.96046712138071, -35.43230574260971),
 (-39.24725960818251, -34.30719441455614),
 (-38.918745689163856, -34.74094846170415),
 (-37.1461175184633, -33.06060066985269),
 (-28.84460557472604, -24.685002151281758),
 (-13.46030365157958, -7.574610362949578),
 (-11.481224950795532, -7.86555447939318),
 (-12.976067006783765, -8.19297049424103),
 (-13.047449655625664, -8.985764371685718),
 (-16.528547258876767, -7.715819001377735),
 (-14.645025015875872, -10.473567560575),
 (-13.477016237986199, -9.173146796263083),
 (-13.277056917969604, -8.664664091305234),
 (-11.691329546834453, -7.463421571947753),
 (-11.045686617919264, -6.935632289492815),
 (-14.667550651946891, -11.340308826025291),
 (-14.245803700750415, -10.288924697353295),
 (-14.408102369885954, -9.668722795591272),
 (-12.894771879424729, -9.741233501431454),
 (-12.12600120682582, -6.844953005763257),
 (-14.172808112135368, -9.722805685055002),
 (-13.248672076968216, -7.79601424061407

In [57]:
eval_source_df

,Distance_km,-39.90625,-39.875,-39.84375,-39.8125,-39.78125,-36.03125,-32.3125,-28.59375,-24.875,...,1.15625,4.875,8.59375,12.3125,16.03125,19.75,23.46875,27.1875,30.90625,34.625
599,2000,-30.952845,-37.577102,-39.467014,-36.413840,-30.867196,-24.313208,-10.744640,-5.859412,-20.457004,...,-9.735727,-15.890355,-12.871940,-11.749949,-11.823074,-6.065181,-8.970509,-10.453740,-28.955520,-42.667507
899,2000,-38.626345,-37.307746,-36.912428,-40.444150,-34.047738,-30.844026,-10.565727,-10.520396,-8.968845,...,-8.097840,-10.399957,-14.284960,-16.429999,-8.238171,-4.324926,-10.216567,-10.120007,-31.599029,-33.838913
2474,2000,-33.779372,-38.394592,-40.804587,-44.127026,-36.087323,-23.410930,-11.121689,-14.142847,-8.028842,...,-5.979173,-7.645291,-14.823989,-5.385098,-12.975748,-12.604770,-9.178475,-17.513869,-22.971308,-37.681188
774,2000,-44.432171,-36.310568,-32.307643,-34.145518,-35.127271,-32.688339,-2.504020,-6.965549,-13.132552,...,-3.295245,-16.031674,-6.317757,-14.963594,-13.902591,-18.410041,-18.194904,-5.473235,-25.179664,-33.874092
1324,2000,-39.587489,-38.265326,-36.933693,-34.628849,-33.055945,-28.743525,-21.978614,-13.635971,-13.057455,...,-8.981007,-15.616872,-15.880854,-9.066092,-12.159647,-13.223600,-13.698687,-16.441145,-26.405733,-35.363971
274,2000,-29.095923,-38.051929,-41.016294,-35.643827,-35.858224,-22.159282,-5.290219,-11.006073,-6.856913,...,-5.501261,-13.442024,-10.367214,-14.516762,-4.838489,-9.680081,-10.387451,-11.548800,-25.750305,-26.686227
724,2000,-53.600671,-36.613207,-29.921509,-38.000470,-37.792198,-29.711215,-11.834105,-7.094779,-7.614989,...,-14.896037,-9.619629,-11.034104,-7.496539,-12.540731,-5.157873,-18.790754,-7.179510,-23.796021,-29.195736
474,2000,-33.965398,-32.461275,-31.833409,-31.514901,-41.158250,-23.341084,-8.354101,-12.777820,-10.932856,...,-8.920163,-14.251563,-8.298995,-16.787533,-12.906668,-9.143870,-12.113604,-8.465622,-27.115329,-38.115949
549,2000,-29.647082,-33.344067,-36.295045,-38.596378,-37.882390,-25.610277,-10.800215,-7.887742,-9.285001,...,-13.237112,-13.091651,-11.616742,-15.160007,-12.033615,-4.770930,-7.572135,-14.865807,-24.054775,-31.234322
1999,2000,-41.061646,-43.638053,-42.280647,-34.783512,-29.157054,-26.826152,-11.981241,-6.843309,-7.510730,...,-11.263030,-14.050281,-17.177087,-8.828553,-11.761292,-11.473498,-10.354983,-3.161696,-26.204797,-30.368733
